In [ ]:
import geopandas as gpd
import numpy as np
import shapely
from shapely import Point, LineString, MultiPoint, MultiLineString

In [ ]:
# Load geoJSON of ICESat-2 tracks over a given site
rgt_file = '/home/jovyan/EverythingAnyWhereAllAtOnce/scripts/tls_beams.geojson'
rgt_df = gpd.read_file(rgt_file, driver='geoJSON')

# Read in the SnowEx lidar boxes
lidar_file = '/home/jovyan/snowex_lidar_swaths.shp'
lidar_df = gpd.read_file(lidar_file, driver='shp')

In [ ]:
# View the available RGTs
print(np.unique(rgt_df['rgt']))

In [ ]:
# Subset RGTs to a desired month
rgt_lines = rgt_df[rgt_df['time_str'].str.contains('Oct')]
rgt_lines

In [ ]:
# Make a quick map to view the available RGTs
m = lidar_df.explore(tiles='Esri.WorldImagery')
rgt_lines.explore(m=m, color='red')

In [ ]:
# Approximate the change in distance between an RGT and the desired TOO point
# WORK IN PROGRESS
dellat = latn - lat0
dellon = lonn - lon0

for i,rgt in enumerate(rgt_lines['rgt']):
    # Loop through each RGT
    tmp = rgt_lines[rgt_lines['rgt']==rgt]
    
    # Process linestrings
    line = tmp[tmp.geom_type=='LineString'].geometry.coords
    dellat = line[1] - lat0
    dellon = line[0] - lon0
    line_too = LineString([Point(t[0]+dellon,t[1]+dellat) for t in line])
    
    # Process points
    point = tmp[tmp.geom_type=='Point'].geometry.coords
    point_too = Point([t[0]+dellon,t[1]+dellat) for t in point])
    
    # Process multipoints - needs looping for account for all beam paths
    for i in [0, 1, 2]:
        multipoint = tmp[tmp.geom_type=='MultiPoint'].geometry.geoms[i].coords
        multipoint_too = MultiPoint([t[0]+dellon,t[1]+dellat) for t in multipoint])